In [8]:
!pip install -q huggingface_hub gradio python-dotenv


In [9]:
import os
from huggingface_hub import InferenceClient

HF_TOKEN = ""
client = InferenceClient(token=HF_TOKEN)




# **PART 1 — WORLD BUILDING (Huggingface API)**

### World Generator Prompt

In [10]:
system_prompt = """
Your job is to help create interesting fantasy worlds.
Rules:
- Plain text only
- Simple language
- 3–5 sentences max
"""


### Generate World

In [ ]:
world_prompt = """
Generate a fantasy world with cities built on the backs of massive beasts.

Format:
World Name: <NAME>
World Description: <DESCRIPTION>

World Name:
"""

response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ]
)

world_output = response.choices[0].message.content.strip()
print(world_output)

world = {
    "name": world_output.split("\n")[0].replace("World Name:", "").strip(),
    "description": "\n".join(world_output.split("\n")[1:]).replace(
        "World Description:", ""
    ).strip()
}


### Generate Kingdoms

In [ ]:
kingdom_prompt = f"""
Create 3 kingdoms.

Format:
Kingdom 1 Name:
Kingdom 1 Description:
Kingdom 2 Name:
Kingdom 2 Description:
Kingdom 3 Name:
Kingdom 3 Description:

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1 Name:
"""

response = client.chat(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ]
)

kingdoms = {}
for block in response.choices[0].message.content.split("\n\n"):
    name = block.split("\n")[0].split("Name:")[1].strip()
    desc = block.split("\n")[1].split("Description:")[1].strip()
    kingdoms[name] = {
        "name": name,
        "description": desc
    }
    print(f"Created kingdom: {name}")

world["kingdoms"] = kingdoms


### Generate Towns

In [ ]:
def create_towns(world, kingdom):
    prompt = f"""
Create 3 towns.

Format:
Town 1 Name:
Town 1 Description:
Town 2 Name:
Town 2 Description:
Town 3 Name:
Town 3 Description:

World: {world['description']}
Kingdom: {kingdom['description']}

Town 1 Name:
"""
    response = client.chat(
        model="meta-llama/Llama-3-8b-chat-hf",
        messages=[{"role": "user", "content": prompt}]
    )

    towns = {}
    for block in response.choices[0].message.content.split("\n\n"):
        name = block.split("\n")[0].split("Name:")[1].strip()
        desc = block.split("\n")[1].split("Description:")[1].strip()
        towns[name] = {
            "name": name,
            "description": desc
        }
        print(f"Town created: {name}")

    kingdom["towns"] = towns


for kingdom in world["kingdoms"].values():
    create_towns(world, kingdom)


### Generate NPCs (1 Town Only)

In [ ]:
def create_npcs(world, kingdom, town):
    prompt = f"""
Create 3 characters.

Format:
Character 1 Name:
Character 1 Description:
Character 2 Name:
Character 2 Description:
Character 3 Name:
Character 3 Description:

World: {world['description']}
Kingdom: {kingdom['description']}
Town: {town['description']}

Character 1 Name:
"""
    response = client.chat(
        model="meta-llama/Llama-3-8b-chat-hf",
        messages=[{"role": "user", "content": prompt}]
    )

npcs = {}
lines = [line for line in block.split("\n") if line.strip()]
if len(lines) < 2:
  desc = block.split("\n")[1].split("Description:")[1].strip()
  npcs[name] = {"name": name, "description": desc}
  print(f"NPC created: {name}")
  town["npcs"] = npcs


first_kingdom = list(world["kingdoms"].values())[0]
for town in first_kingdom["towns"].values():
    create_npcs(world, first_kingdom, town)


### Save World

In [ ]:
import json

with open("YourWorld.json", "w") as f:
    json.dump(world, f, indent=2)

print("World saved!")


# **PART 2 — GAMEPLAY (Groq API)**

### Load World & Start Story

In [ ]:
with open("YourWorld.json") as f:
    world = json.load(f)

kingdom = list(world["kingdoms"].values())[0]
town = list(kingdom["towns"].values())[0]
character = list(town["npcs"].values())[0]

prompt = f"""
Create the start of an adventure.
Rules:
- 2–4 sentences
- Second person
- Present tense

Character: {character['description']}
Town: {town['description']}
"""

response = client.chat(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[{"role": "user", "content": prompt}]
)

start = response.choices[0].message.content
print(start)


### Game State

In [ ]:
game_state = {
    "world": world["description"],
    "kingdom": kingdom["description"],
    "town": town["description"],
    "character": character["description"],
    "start": start,
    "inventory": {
        "cloth shirt": 1,
        "cloth pants": 1,
        "gold": 5
    }
}


# **PART 3 — SAFETY CHECK**

In [ ]:
def is_safe(text):
    prompt = f"""
You are a content moderator.
Answer ONLY safe or unsafe.

Text:
{text}
"""
    response = client.chat(
        model="meta-llama/Llama-3-8b-chat-hf",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content.strip().lower() == "safe"


# **PART 4 — INVENTORY SYSTEM (Together API)**

### Inventory Detection

In [ ]:
def detect_inventory_change(story_text):
    prompt = f"""
You are an inventory system for a text RPG.

Rules:
- Return ONLY valid JSON
- Keys = item names
- Values = integers (positive = gained, negative = lost)
- If no change, return {{}}

Story:
{story_text}

JSON:
"""
    response = client.chat(
        model="meta-llama/Llama-3-8b-chat-hf",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
import json

try:
    return json.loads(response.choices[0].message.content.strip())
except json.JSONDecodeError:
    return {}


### Inventory Update Logic

In [ ]:
def update_inventory(inventory, changes):
    for item, amount in changes.items():
        inventory[item] = inventory.get(item, 0) + amount
        if inventory[item] <= 0:
            del inventory[item]


# **PART 5 — FINAL GAME LOOP + UI**

### Game Loop

In [ ]:
def run_action(message, history, game_state):
    if message.lower() == "start game":
        return game_state["start"]

    system_prompt = """You are an AI Game Master.
Rules:
- 1–3 sentences
- Second person
- Present tense
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Character: {game_state['character']}
Inventory: {game_state['inventory']}
"""}
    ]

    for u, a in history:
        messages.append({"role": "assistant", "content": a})
        messages.append({"role": "user", "content": u})

    messages.append({"role": "user", "content": message})

    response = client.chat(
        model="meta-llama/Llama-3-8b-chat-hf",
        messages=messages
    )

    story = response.choices[0].message.content

    changes = detect_inventory_change(story)
    update_inventory(game_state["inventory"], changes)

    return story


### Show Inventory

### Launch Gradio Game

In [ ]:
import gradio as gr

def main_loop(message, history):
    if not is_safe(message):
        return "⚠️ Action not allowed."

    output = run_action(message, history, game_state)

    if not is_safe(output):
        return "⚠️ Story blocked."

    inventory_text = "\n\n🎒 Inventory:\n"
    for item, qty in game_state["inventory"].items():
        inventory_text += f"- {item}: {qty}\n"

    return output + inventory_text


In [ ]:
# Click Gradio public link
# Type: start game